# Calling in Parallel

  By Kyle Baacke
  3/13/2022

## Description:
  You may find yourself wanting to run a process in parallel (i.e. multiple processes running simultaneously) via the native shell (or zshell) native to your computational environment. This snippet provides an example of how to do so without the use of any non-generic packages or functions. This can enable you to make full use of the compute resources available without the additional overhead of installing other tools. The process can be divided into 3 main steps: 1) establishing the resource pool available to use, 2) defining a function to run in parallel if there are multipe components of a process that need to be run for each iteration, and 3) calling the function in parallel. This snippit will go through each process with an example which callas a python script using fariables in nested for loops.
    
## Notes and Qualifiers:
  This code has been written and tested in BASH on native linux installations (OpenSUSE) and on Ubuntu for Windows (20.04.2 using WSL 2). 
  
## 1) Evaluate Resource Pool
  Before you run anyting in parallel, you should first establish how many CPU threads ara avialble for use. The following command will give you details about the CPUs on the current machine. In this case, 12 is the nu,ber we are looking for (line 5):

In [ ]:
lscpu

#1 Architecture:                    x86_64
#2 CPU op-mode(s):                  32-bit, 64-bit
#3 Byte Order:                      Little Endian
#4 Address sizes:                   48 bits physical, 48 bits virtual
#5 CPU(s):                          12
# On-line CPU(s) list:             0-11
# Thread(s) per core:              2
# Core(s) per socket:              6
# Socket(s):                       1
# Vendor ID:                       AuthenticAMD
# CPU family:                      23
# Model:                           8
# Model name:                      AMD Ryzen 5 2600X Six-Core Processor
# Stepping:                        2
# CPU MHz:                         3600.265
# BogoMIPS:                        7200.53
# Hypervisor vendor:               Microsoft
# Virtualization type:             full
# L1d cache:                       192 KiB
# L1i cache:                       384 KiB
# L2 cache:                        3 MiB
# L3 cache:                        8 MiB
# Vulnerability Itlb multihit:     Not affected
# Vulnerability L1tf:              Not affected
# Vulnerability Mds:               Not affected
# Vulnerability Meltdown:          Not affected
# Vulnerability Spec store bypass: Mitigation; Speculative Store Bypass disabled via prctl and seccomp
# Vulnerability Spectre v1:        Mitigation; usercopy/swapgs barriers and __user pointer sanitization
# Vulnerability Spectre v2:        Mitigation; Full generic retpoline, IBPB conditional, STIBP disabled, RSB filling
# Vulnerability Srbds:             Not affected
# Vulnerability Tsx async abort:   Not affected
# Flags:                           fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxs
#                                  r sse sse2 ht syscall nx mmxext fxsr_opt pdpe1gb rdtscp lm rep_good nopl cpuid extd_api
#                                  cid pni pclmulqdq ssse3 fma cx16 sse4_1 sse4_2 movbe popcnt aes xsave avx f16c rdrand h
#                                  ypervisor lahf_lm cmp_legacy cr8_legacy abm sse4a misalignsse 3dnowprefetch osvw topoex
#                                  t ssbd ibpb vmmcall fsgsbase bmi1 avx2 smep bmi2 rdseed adx smap clflushopt sha_ni xsav
#                                  eopt xsavec xgetbv1 xsaves clzero xsaveerptr virt_ssbd arat

  Always make sure you will not cause problems for other users by checking to see if anyone else is currenlty using the same machine. You can do this and learn more about the current memory and thread availability using either of the following commands. This will give you an active diplay of the processes currently running on the machine, including CPU and memory usage and runtime. Use q to exit the top display.

In [ ]:
top
# or
htop

## 2) Defining your function
  Now that you know what you have to work with, you need to set up a function to run in parallel. The following function takes three arguements (subject, time, and run, in that order). It then uses these arguements to call a python function (nifty2numpy_parcellation.py). First, lets start by instantiating some global constant variables.

In [ ]:
OUTDIR2=/mnt/usb1/HCP_1200_ICA-AROMA
TIME=tfMRI_MOTOR
# We need a list of subjects to iterate through. The following command creates a list out of each folder in the directory we assigned to OUTDIR2
SUBLIST=$(ls $OUTDIR2)
# Now assign the number of threads to use as a variable, leaving some for other processes.
N=10

  The following script creates a function. This function will take assign the first argument after the function call and assign it to the *subject* variable fot that function run. The *time* and *run* variables are assigned on a per-run basis in the same way using arguments *2* and *3*.

In [ ]:
parcellate(){
  # Use $ fllowed by a number to take an arguement passed to the function and assign it to a variable
  local subject=$1
  local time=$2
  local run=$3
  # This prints the target path to the terminal so we know which image we are targetting. In the event that re run into an error, we can check this against the true path as a way to check if the path is correct
  echo ${OUTDIR2}/${subject}/MNINonLinear/Results/${time}_${run}/${time}_${run}-AROMA-denoised-nonsmoothed.nii.gz
  python3 nifty2numpy_parcellation.py \
    -input ${OUTDIR2}/${subject}/MNINonLinear/Results/${time}_${run}/${time}_${run}-AROMA-denoised-nonsmoothed.nii.gz \
    -atlas Schaefer2018_200Parcels_7Networks_order_FSLMNI152_2mm \
    --metadata /mnt/usb1/Code/MSC_HCP/
}

## 3) Calling in parallel
  Then, we can iterate through the subjects in *SUBLIST* and the runs within each subject using nested for loops.

In [ ]:
for SUBJECT in $SUBLIST; do
  for RUN in RL LR; do
    # First we call the function for a SUBJECT, RUN to start one process.
    parcellate "$SUBJECT" "$TIME" "$RUN" &
    # Then we check to see if the number of processes we have running matches the $N value that we set
    if [[ $(jobs -r -p | wc -l) -ge $N ]]; then
      # If we are at our allotted ammount, then we will wait until one of our jobs is complete before starting another
      wait -n
    fi
   done
done

Note that you can keep 'htop' running in a second window to track your use of resources in real time. This is a good way to keep track of how far along you are, as well as what bottlenecks you have when running N threads at a time. For instance, you may find that you can only run 4 at a time, even through you have 12 threads because of a memory constraint.